In [1]:
import pandas as pd
import os
import functools
pd.options.mode.chained_assignment = None

In [2]:
def write_df_excel(df: pd.DataFrame, sheet_name: str, create_writer = False, close = False) -> None:
    """
    Write a single DataFrame to an excel file.
    """
    global writer 
    if create_writer:
        writer = pd.ExcelWriter(f"results/razor_protein.xlsx")
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    if close:
        writer.close()

In [3]:
def write_df_excel_wrap(func):
    """
    Wrap another function that returns a DataFrame and write it to an excel file.
    """
    @functools.wraps(func)
    def capture(*args, **kwargs):
        result_df: pd.DataFrame = func(*args, **kwargs)   
        write_df_excel(result_df, func.__name__)
        return result_df
    return capture

In [4]:
def FiltPeps(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filter for 5% FDR, remove decoy proteins, and extract peptide sequence 
    """
    qvalue_filtered_df = df[df["QValue"] <= 0.05]
    non_decoy_filtered_df = qvalue_filtered_df[~qvalue_filtered_df["Protein"].str.startswith("XXX")]
    non_decoy_filtered_df["PeptideSequence"] = non_decoy_filtered_df["Peptide"].str.extract(r"\.(.*)\.")
    return non_decoy_filtered_df

In [5]:
def get_FiltPeps(df: pd.DataFrame) -> pd.DataFrame:
    """
    Get filtered peptide results and return DataFrame in a shape we want to use
    """
    FiltPeps_df = FiltPeps(df)
    return (
        FiltPeps_df[["PeptideSequence", "Protein"]]
        .drop_duplicates()
        .sort_values(by=["PeptideSequence"])
    )

In [6]:
# Read resultant .tsv
resultant_df = pd.read_csv("SpruceW_P19_15_22Jun17_Pippin_17-04-06_msgfplus_syn_PlusSICStats.txt", sep="\t", float_precision="round_trip")

#write_df_excel(resultant_df, "1, resultant", True)

In [7]:
# Get filtered peptide <-> protein table. (query_1)
temp_df = resultant_df.copy()
filt_peps_df = get_FiltPeps(temp_df).drop_duplicates().sort_values(by=["PeptideSequence"])

#write_df_excel(filt_peps_df, "2, filtered 0.05 fdr")

In [8]:
# count all peptides per protein, flatten results. (query_2)
temp_df = resultant_df.copy()
FiltPeps_df = get_FiltPeps(temp_df)
grouped_by_protein = FiltPeps_df.groupby("Protein")
counted_peptides_df = grouped_by_protein["PeptideSequence"].count().reset_index(name="peptides_per_protein")

#write_df_excel(counted_peptides_df, "3, peptides_per_protein")

In [9]:
# count all proteins per peptides, flatten results. (query_3)
temp_df = resultant_df.copy()
FiltPeps_df = get_FiltPeps(temp_df)
grouped_by_peptide = FiltPeps_df.groupby("PeptideSequence")
counted_protein_df = grouped_by_peptide["Protein"].count().reset_index(name="proteins_per_peptide")

#write_df_excel(counted_protein_df, "4, proteins_per_peptide")

In [10]:
# merge the results into a filtered master list for us to work with. (query_4)
merge_1 = filt_peps_df.merge(counted_peptides_df, how="inner", on="Protein")
merge_3 = merge_1.merge(counted_protein_df, how="inner", on="PeptideSequence")
col_of_interest = ["Protein", "PeptideSequence", "peptides_per_protein", "proteins_per_peptide"]
peptide_protein_map_with_count_df = merge_3[col_of_interest].sort_values(by=["PeptideSequence"])

#write_df_excel(peptide_protein_map_with_count_df, "5, merge")

print(peptide_protein_map_with_count_df)

                               Protein         PeptideSequence  \
0      nmdc:mga0wn63_328416_c1_556_681            AAAAAKSEAAPK   
1        nmdc:mga0wn63_731999_c1_3_410            AAAADAGQPLWR   
2     nmdc:mga0wn63_164_c1_30348_31634            AAAADYGLPLYR   
3        nmdc:mga0wn63_431891_c1_2_571         AAAANSDQQLQQAVR   
4        nmdc:mga0wn63_951089_c1_1_342  AAAAQAAGYFTEQILPVEVAGK   
...                                ...                     ...   
5680  nmdc:mga0wn63_143199_c1_253_1158              YYQDNLDQFK   
5681    nmdc:mga0wn63_2243_c1_374_4312           YYQTQLYAQDTWK   
5682   nmdc:mga0wn63_5268_c1_5907_6251               YYRLTAAGR   
5683  nmdc:mga0wn63_14203_c1_2827_4140                 YYTPSGR   
5684     nmdc:mga0wn63_74401_c1_1_1581                 YYTPSGR   

      peptides_per_protein  proteins_per_peptide  
0                        1                     1  
1                        1                     1  
2                        2                     1  
3  

In [11]:
# PEPTIDE <-> RAZOR PROTEIN ASSOCIATION

In [12]:
# First case
peps_with_1_protein = peptide_protein_map_with_count_df[peptide_protein_map_with_count_df["proteins_per_peptide"] == 1]

# FIRST COLLECTION OF PEPTIDE <->  RAZOR PROTEIN RELATIONSHIPS
non_degenerate_razor_protein_df = peps_with_1_protein[["PeptideSequence", "Protein"]]

#write_df_excel(non_degenerate_razor_protein_df, "6, prots_per_peps == 1 -> RP")
print(non_degenerate_razor_protein_df)

             PeptideSequence                           Protein
0               AAAAAKSEAAPK   nmdc:mga0wn63_328416_c1_556_681
1               AAAADAGQPLWR     nmdc:mga0wn63_731999_c1_3_410
2               AAAADYGLPLYR  nmdc:mga0wn63_164_c1_30348_31634
3            AAAANSDQQLQQAVR     nmdc:mga0wn63_431891_c1_2_571
4     AAAAQAAGYFTEQILPVEVAGK     nmdc:mga0wn63_951089_c1_1_342
...                      ...                               ...
5676           YYGHRYYRGYWHR  nmdc:mga0wn63_35292_c1_1633_1878
5677               YYLQKVVPK    nmdc:mga0wn63_1020302_c1_2_337
5680              YYQDNLDQFK  nmdc:mga0wn63_143199_c1_253_1158
5681           YYQTQLYAQDTWK    nmdc:mga0wn63_2243_c1_374_4312
5682               YYRLTAAGR   nmdc:mga0wn63_5268_c1_5907_6251

[4717 rows x 2 columns]


In [13]:
# Get table that lists multiple peptide sequences, we want all peptides that are associated with MULTIPLE proteins
peps_with_many_proteins = peptide_protein_map_with_count_df[peptide_protein_map_with_count_df["proteins_per_peptide"] > 1]
degenerate_peptide_list = peps_with_many_proteins[["PeptideSequence", "Protein"]]

#write_df_excel(peps_with_many_proteins, "7, prots_per_peps > 1")

# - find peptides where more than one NonDegereate_razor_protein_list member is associated (get all peptides in NonNonDegereate_razor_protein_list [a])
#   - merge Degenerate_peptide_list with NonDegenerate_razor_protein_list on peptide (take table 4) //(all peptides with protein count > 1 where the protein is in NonDegereate_razor_protein_list)
#   - group by peptide and count NonDegenerate_razor_protein_list members -> NonDegenerate_razor_protein_list_member_count (peptide, proteins_per_peptide)
# Remove ALL proteins NOT found in Non-Degen set
non_degenerate_protein_set = set(non_degenerate_razor_protein_df['Protein'].unique())
NonDegenerate_razor_protein_list_member_int_df = peps_with_many_proteins[peps_with_many_proteins['Protein'].isin(non_degenerate_protein_set)]

#write_df_excel(NonDegenerate_razor_protein_list_member_int_df, "8, '7' in non-degen set")

In [14]:
#- for NonDegenerate_razor_protein_list_member_count = 1 -> Degenerate_peptides_with_only_unique_protein (peptide, razor_protein)
# We have grouped on peptide for the set of proteins_per_peptide > 1 and removed any rows whose Proteins are NOT in the non-degen set
# therefore, we only need to look for unique entries to find protein to razor protein mapping for this collection of degen peptides

# SECOND COLLECTION OF PEPTIDE <->  RAZOR PROTEIN RELATIONSHIPS
Degenerate_peptides_with_only_unique_protein =  NonDegenerate_razor_protein_list_member_int_df.drop_duplicates(subset=['PeptideSequence'], keep=False, inplace=False)[['PeptideSequence', 'Protein']]

#write_df_excel(Degenerate_peptides_with_only_unique_protein, "9, '8' but unique -> RP")
print(Degenerate_peptides_with_only_unique_protein)

        PeptideSequence                           Protein
73    AAVEEGIVPGGGVALAR  nmdc:mga0wn63_379_c1_32789_34459
171         AFEKDVLPLLR     nmdc:mga0wn63_334658_c1_1_273
180         AFFFGAYEGQR    nmdc:mga0wn63_43302_c1_94_2394
506        AVAAGLNPMDLK     nmdc:mga0wn63_435591_c1_3_536
536         AVGELMENQFR    nmdc:mga0wn63_9_c1_95095_99576
...                 ...                               ...
5507          YFGPSTLDR    nmdc:mga0wn63_3933_c1_950_4651
5627        YTGEPISVNLK    nmdc:mga0wn63_109129_c1_3_1373
5660       YVQATEAPIQAK   nmdc:mga0wn63_3987_c1_4243_4986
5678    YYLTTDGPNKVDASR   nmdc:mga0wn63_37592_c1_469_2136
5683            YYTPSGR  nmdc:mga0wn63_14203_c1_2827_4140

[77 rows x 2 columns]


In [15]:
# - for NonDegenerate_razor_protein_list_member_count > 1 -> Degenerate_peptides_with_morethanone_unique_protein (peptide, unique_protein_count)
# - remove peptide from filter passing peptides that match Degenerate_peptides_with_morethanone_unique_protein (filter passing being table_4?, remove all matching peptide instances or just peptide-prot pair?)
# We now take the same base set as above but look for the NON-UNIQUE degen peptides in that set. These ones we can't make determinations about
# since they're associated with MORE THAN ONE protein found in the non-degen protein set and will be removed from the set of filtered peptides.
Degenerate_peptides_with_morethanone_unique_protein_to_remove_df = NonDegenerate_razor_protein_list_member_int_df[NonDegenerate_razor_protein_list_member_int_df['PeptideSequence'].duplicated(keep=False)]

#write_df_excel(Degenerate_peptides_with_morethanone_unique_protein_to_remove_df, "10, '8' dups to remove")
print(Degenerate_peptides_with_morethanone_unique_protein_to_remove_df)

                               Protein               PeptideSequence  \
908       nmdc:mga0wn63_84419_c1_3_629                 DYLVTDKGIDASR   
907      nmdc:mga0wn63_156896_c1_3_971                 DYLVTDKGIDASR   
1024    nmdc:mga0wn63_9_c1_95095_99576      EFFGSSQLSQFMDQTNPLSEITHK   
1025     nmdc:mga0wn63_725835_c1_3_413      EFFGSSQLSQFMDQTNPLSEITHK   
1027    nmdc:mga0wn63_131868_c1_2_1219      EFFGSSQLSQFMDQTNPLSEITHK   
1167   nmdc:mga0wn63_313_c1_8773_12285                  EINTLPLASTER   
1168     nmdc:mga0wn63_340107_c1_3_665                  EINTLPLASTER   
1275    nmdc:mga0wn63_1488111_c1_1_252             EQVTADLQGGVYKVPGR   
1274    nmdc:mga0wn63_1091877_c1_2_325             EQVTADLQGGVYKVPGR   
1536            Contaminant_K2C1_HUMAN                  FLEQQNQVLQTK   
1537            Contaminant_K22E_HUMAN                  FLEQQNQVLQTK   
2126    nmdc:mga0wn63_1251082_c1_1_303  IGDLILEHLDELAQLESLDNGKPFAVAR   
2127     nmdc:mga0wn63_26396_c1_2_1000  IGDLILEHLDELAQLESLDNGKPF

In [16]:
# - find max peptide_per_protein_count for all remaining proteins grouped to peptide in question (remove all found peptides with a razor protein from master list?)
# - store this as a value
# - retrieve all proteins that match this stored value for peptide in question -> Degenerate_peptides_with_max_nonunique_peptide_count (peptide, razor_protein)
# - practical effect: if one protein has max number or if there is a list of proteins with this max number, both situations pass

max_peptide_per_protein_count = peps_with_many_proteins.copy()
non_degenerate_peptide_set_to_remove = set(non_degenerate_razor_protein_df['PeptideSequence'])
Degenerate_peptides_with_only_unique_protein_to_remove = set(Degenerate_peptides_with_only_unique_protein['PeptideSequence'])
Degenerate_peptides_with_morethanone_unique_protein_to_remove_set = set(Degenerate_peptides_with_morethanone_unique_protein_to_remove_df['PeptideSequence'])

# Remove all matched peptides and those we think can't be matched from filtered set of mappings
max_peptide_per_protein_count = max_peptide_per_protein_count[~max_peptide_per_protein_count['PeptideSequence'].isin(Degenerate_peptides_with_morethanone_unique_protein_to_remove_set)]
max_peptide_per_protein_count = max_peptide_per_protein_count[~max_peptide_per_protein_count['PeptideSequence'].isin(Degenerate_peptides_with_only_unique_protein_to_remove)]
max_peptide_per_protein_count = max_peptide_per_protein_count[~max_peptide_per_protein_count['PeptideSequence'].isin(non_degenerate_peptide_set_to_remove)]

# find the max peptides_per_protein for a grouped set of peptides, then flatten the result
max_peptide_per_protein_count["max_peptides_per_protein"] = max_peptide_per_protein_count.groupby(["PeptideSequence"])["peptides_per_protein"].transform("max")

#write_df_excel(max_peptide_per_protein_count, "11, prots_per_peps > 1 - found")


# Grab the peptide-protein pairs that have max peptides_per_protein.
# the relationship will be at least:
#    - one-to-one: peptide to maximally associated protein.
#    - one-to-many: peptide to ALL PROTEINS whose MAX peptides_per_protein == all other max values (including 1) and are the largest
#      in the set of peptides_per_protein values for all proteins grouped for a unique peptide.
max_peptide_per_protein_count_df = max_peptide_per_protein_count[max_peptide_per_protein_count["peptides_per_protein"] == max_peptide_per_protein_count["max_peptides_per_protein"]]
#write_df_excel(max_peptide_per_protein_count, "12, peps_per_prots == max")

# THIRD COLLECTION OF PEPTIDE <-> RAZOR PROTEIN RELATIONSHIPS
Degenerate_peptides_with_max_nonunique_peptide_count_df = max_peptide_per_protein_count[["PeptideSequence", "Protein"]]

#write_df_excel(Degenerate_peptides_with_max_nonunique_peptide_count_df, "13, '12' -> RP")
print(Degenerate_peptides_with_max_nonunique_peptide_count_df)

        PeptideSequence                            Protein
15         AADVPLAVDLFR    nmdc:mga0wn63_376314_c1_207_623
16         AADVPLAVDLFR    nmdc:mga0wn63_39782_c1_171_1811
17         AADVPLAVDLFR    nmdc:mga0wn63_54_c2_18859_20364
38    AAIEEGVVPGGGVALLR    nmdc:mga0wn63_3548_c1_1950_3569
39    AAIEEGVVPGGGVALLR      nmdc:mga0wn63_989718_c1_2_343
...                 ...                                ...
5642       YTWNGEITGLFK    nmdc:mga0wn63_3686_c1_7397_8134
5650         YVESLSAYAR  nmdc:mga0wn63_1774_c1_13878_15950
5649         YVESLSAYAR    nmdc:mga0wn63_39094_c1_630_2546
5648         YVESLSAYAR    nmdc:mga0wn63_40181_c1_934_2508
5647         YVESLSAYAR    nmdc:mga0wn63_482949_c1_204_530

[714 rows x 2 columns]


In [17]:
# Combine all 
all_peptides_to_razor_protein_map_df = pd.concat([non_degenerate_razor_protein_df, Degenerate_peptides_with_only_unique_protein, Degenerate_peptides_with_max_nonunique_peptide_count_df])
all_peptides_to_razor_protein_map_df = all_peptides_to_razor_protein_map_df.rename(columns={"Protein": "BestProtein"})

#write_df_excel(all_peptides_to_razor_protein_map_df, "14, ALL RAZOR PROTEINS", False, True)
print(all_peptides_to_razor_protein_map_df)

             PeptideSequence                        BestProtein
0               AAAAAKSEAAPK    nmdc:mga0wn63_328416_c1_556_681
1               AAAADAGQPLWR      nmdc:mga0wn63_731999_c1_3_410
2               AAAADYGLPLYR   nmdc:mga0wn63_164_c1_30348_31634
3            AAAANSDQQLQQAVR      nmdc:mga0wn63_431891_c1_2_571
4     AAAAQAAGYFTEQILPVEVAGK      nmdc:mga0wn63_951089_c1_1_342
...                      ...                                ...
5642            YTWNGEITGLFK    nmdc:mga0wn63_3686_c1_7397_8134
5650              YVESLSAYAR  nmdc:mga0wn63_1774_c1_13878_15950
5649              YVESLSAYAR    nmdc:mga0wn63_39094_c1_630_2546
5648              YVESLSAYAR    nmdc:mga0wn63_40181_c1_934_2508
5647              YVESLSAYAR    nmdc:mga0wn63_482949_c1_204_530

[5508 rows x 2 columns]
